<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_tachyon_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_tachyon_ai.py

End-to-end training script for TachyonAI:
  - Defines an MLP (TachyonAI) for tachyonic field evolution
  - Generates synthetic (energy, momentum, field strength) → (next_amplitude, next_rate)
  - Trains with MSE, LR scheduler, early stopping
  - Evaluates on a held-out validation set
  - Plots true vs. predicted field evolution
"""

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import TensorDataset, DataLoader, random_split
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------
# 1. Model Definition
# ------------------------------------------------------------------------------
class TachyonAI(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TachyonAI, self).__init__()
        self.fc1  = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2  = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        return self.fc2(x)

# ------------------------------------------------------------------------------
# 2. Synthetic Data Generator
#    Toy dynamical rule for tachyonic field evolution:
#      Given (E, p, f) at time t, predict (f_{t+Δ}, df/dt_{t+Δ})
# ------------------------------------------------------------------------------
def generate_synthetic_tachyon_data(n_samples=12000, seed=42):
    np.random.seed(seed)
    # Sample energy E, momentum p, field strength f in [-1,1]
    E = np.random.uniform(-1, 1, size=(n_samples, 1)).astype(np.float32)
    p = np.random.uniform(-1, 1, size=(n_samples, 1)).astype(np.float32)
    f = np.random.uniform(-1, 1, size=(n_samples, 1)).astype(np.float32)

    X = np.hstack([E, p, f])  # shape (n_samples, 3)

    # Toy evolution laws:
    #   f_next = f + Δt * (p - E * f)
    #   rate   = p * f - E
    Δt = 0.1
    f_next = f + Δt * (p - E * f)
    rate   = p * f - E

    Y = np.hstack([f_next, rate]).astype(np.float32)  # shape (n_samples, 2)
    return X, Y

# ------------------------------------------------------------------------------
# 3. Prepare Dataset and DataLoaders
# ------------------------------------------------------------------------------
# Generate and wrap data
X, Y = generate_synthetic_tachyon_data(n_samples=12000)
X_tensor = torch.from_numpy(X)
Y_tensor = torch.from_numpy(Y)

dataset = TensorDataset(X_tensor, Y_tensor)
train_size = int(0.8 * len(dataset))
val_size   = len(dataset) - train_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
train_loader    = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader      = DataLoader(val_ds,   batch_size=256)

# ------------------------------------------------------------------------------
# 4. Instantiate Model, Loss, Optimizer, Scheduler
# ------------------------------------------------------------------------------
device   = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model    = TachyonAI(input_dim=3, hidden_dim=32, output_dim=2).to(device)
criterion= nn.MSELoss()
optimizer= optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler= optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5
)

# Early stopping parameters
best_val_loss = float('inf')
patience      = 0
max_patience  = 10

# ------------------------------------------------------------------------------
# 5. Training Loop
# ------------------------------------------------------------------------------
train_losses, val_losses = [], []

for epoch in range(1, 201):
    # --- Training phase ---
    model.train()
    running_train_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss  = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item() * xb.size(0)

    train_loss = running_train_loss / len(train_loader.dataset)
    train_losses.append(train_loss)

    # --- Validation phase ---
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds  = model(xb)
            running_val_loss += criterion(preds, yb).item() * xb.size(0)

    val_loss = running_val_loss / len(val_loader.dataset)
    val_losses.append(val_loss)

    scheduler.step(val_loss)

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience = 0
        torch.save(model.state_dict(), "best_tachyon_ai.pt")
    else:
        patience += 1
        if patience >= max_patience:
            print(f"Early stopping at epoch {epoch}")
            break

    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | Train MSE: {train_loss:.4f} | Val MSE: {val_loss:.4f}")

print(f"\nTraining complete. Best Val MSE: {best_val_loss:.4f}")

# ------------------------------------------------------------------------------
# 6. Load Best Model & Evaluate on Validation Set
# ------------------------------------------------------------------------------
model.load_state_dict(torch.load("best_tachyon_ai.pt"))
model.eval()

# Collect predictions and true values
all_preds, all_true = [], []
with torch.no_grad():
    for xb, yb in val_loader:
        xb = xb.to(device)
        pred = model(xb).cpu().numpy()
        all_preds.append(pred)
        all_true.append(yb.numpy())

all_preds = np.vstack(all_preds)
all_true  = np.vstack(all_true)

# ------------------------------------------------------------------------------
# 7. Plot True vs. Predicted Evolution
# ------------------------------------------------------------------------------
plt.figure(figsize=(12,5))

for i, label in enumerate(["f_next", "rate"]):
    plt.subplot(1, 2, i+1)
    plt.scatter(all_true[:, i], all_preds[:, i], s=5, alpha=0.3)
    m = np.min([all_true[:, i].min(), all_preds[:, i].min()])
    M = np.max([all_true[:, i].max(), all_preds[:, i].max()])
    plt.plot([m, M], [m, M], 'r--')
    plt.xlabel(f"True {label}")
    plt.ylabel(f"Predicted {label}")
    plt.title(f"{label}: True vs. Predicted")

plt.tight_layout()
plt.show()

# ------------------------------------------------------------------------------
# 8. Loss Curve Plot
# ------------------------------------------------------------------------------
plt.figure()
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses,   label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("Training & Validation Loss")
plt.legend()
plt.show()